<a href="https://colab.research.google.com/github/SREYAKUKUTAPU/Pyspark/blob/main/udf1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("show").getOrCreate()

data=[(1,'sreya',2000,200),(2,'minny',3000,300)]
schema=['id','name','salary','bonus']
df=spark.createDataFrame(data,schema)
df.show()

+---+-----+------+-----+
| id| name|salary|bonus|
+---+-----+------+-----+
|  1|sreya|  2000|  200|
|  2|minny|  3000|  300|
+---+-----+------+-----+



In [2]:
def totalPay(s,b):
  return s+b

In [3]:
totalPayment=udf(lambda s,b : totalPay(s,b),IntegerType())


In [5]:
df.withColumn('totalPay',totalPayment(df.salary,df.bonus)).show()

+---+-----+------+-----+--------+
| id| name|salary|bonus|totalPay|
+---+-----+------+-----+--------+
|  1|sreya|  2000|  200|    2200|
|  2|minny|  3000|  300|    3300|
+---+-----+------+-----+--------+



# Another way of registering udf

In [9]:
@udf(returnType=IntegerType()) #Annotation
def totalPay(s,b):
  return s+b

In [10]:
df.select('*',totalPay(df.salary,df.bonus).alias('totalPay')).show()

+---+-----+------+-----+--------+
| id| name|salary|bonus|totalPay|
+---+-----+------+-----+--------+
|  1|sreya|  2000|  200|    2200|
|  2|minny|  3000|  300|    3300|
+---+-----+------+-----+--------+



# spark.udf.register()

In [13]:
def totalPay(s,b):
  return s+b*4

In [14]:
spark.udf.register(name='TotalPaySQL',f=totalPay,returnType=IntegerType())

<function __main__.totalPay(s, b)>

In [15]:
df.createOrReplaceTempView('employees')

In [16]:
spark.sql('select *,TotalPaySQL(salary,bonus) as totalPay from employees').show()

+---+-----+------+-----+--------+
| id| name|salary|bonus|totalPay|
+---+-----+------+-----+--------+
|  1|sreya|  2000|  200|    2800|
|  2|minny|  3000|  300|    4200|
+---+-----+------+-----+--------+

